# CaCO3 structures

In [1]:
from ase.spacegroup import crystal

a =  4.99
c = 17.0615
caco3 = crystal('CaCO3', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.25), 
                          (0.2578, 0.0, 0.25)], 
                spacegroup='R -3 c', 
                cellpar=[a,a,c,90,90,120])

In [5]:
from ase.visualize import view
view(caco3)

In [17]:
from ase.io import read
caco3_cif = read("caco3_r3c.cif")
view(caco3_cif)

# NaNO3 structures

In [13]:
from ase.spacegroup import crystal
from ase.visualize import view
# T = 541K
a =  5.082
c = 17.31
nano3_low = crystal('NaNO', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.25), 
                          (0.2381, 0.0, 0.25)], 
                spacegroup='R -3 c', 
                cellpar=[a,a,c,90,90,120])

print "Low Temp: T = 541K ", nano3_low.info

# T = 563K

a =  5.084
c = 8.175
nano3_high = crystal('NaNO', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.5), 
                          (0.2337, 0.0, 0.5)], 
                spacegroup='R -3 m', 
                cellpar=[a,a,c,90,90,120])

print "High Temp: T = 563K ", nano3_high.info

Low Temp: T = 541K  {'spacegroup': Spacegroup(167, setting=1), 'unit_cell': 'conventional'}
High Temp: T = 563K  {'spacegroup': Spacegroup(166, setting=1), 'unit_cell': 'conventional'}


In [31]:
view(nano3_low)

In [4]:
view(nano3_high)


array([11, 11, 11, 11, 11, 11,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8])

# NaNO3 MD model

In [29]:
class McDonaldNO3(object):
    def __init__(self, supercell=[1,1,1], cif_file=None):
        self.description = "Sodim Nitrate model for crystalline odered-disordered transition"
        self.citation = { "authors" : [ {"Last" : "Lynden-Bell", "First" : "R M"},
                                        {"Last" : "Ferrario", "First" : "M "},
                                        {"Last" : "McDonald", "First" : "I R"} ],
                          "journal" : "Journal of Physics: Condensed Matter",
                          "year" : 1989,
                          "volume" : 1,
                          "pages" : 6523-6542
                        }
        self._supercell = supercell
        
        self.mol_ids = list()
        self.molecule_counter = 0
        
        if cif_file:
            self._unit_cell = read(cif_file)
        else:
            self._unit_cell = nano3_low = crystal('NaNO', [(0.0,0.0,0.0), 
                                                           (0.0, 0.0, 0.25), 
                                                           (0.2381, 0.0, 0.25)], 
                                             spacegroup='R -3 c', 
                                             cellpar=[a,a,c,90,90,120])
        def _makeSuperCell(self):
            lat = self._lat
            sc  = self._supercell
            self.system = Atoms(cell=[sc[0]*lat,sc[1]*lat,sc[2]*lat], pbc=[1,1,1])
            
    def lammpsParameters(self):
        # Simulation Parameters

        eps_ff = 70.60 # units: K
        sig_ff =  2.70 # units: Angstrom

        eps_ff *= units.kB / (units.kcal/units.mol) # units: kcal/mol

        #print(eps_ff,sig_ff)

        # LAMMPS input
        header = ['units real',
                  'atom_style molecular',
                  'bond_style harmonic',
                  'atom_modify map array sort 0 0']

        cmds = ["pair_style lj/cut 10.0",
                "pair_coeff * * 0.0 0.0",
                "pair_coeff 2 2 "+str(eps_ff)+' '+str(sig_ff),
                "bond_coeff * 100.0 1.565",
                "fix 1 all nve",
                "fix 1 all rigid/nve/small molecule"]

        return header, cmds


In [30]:
from ase.calculators.lammpslib import write_lammps_data
nano3 = NaNO3(supercell=[1,1,1])
header, cms = nano3.lammpsParameters()
system = nano3
write_lammps_data('config.data',
                  molecules,
                  {'Na': 1, 'N': 2, 'O': 3},
                  cutoff=,
                  bond_types=[(7,8)],
                  angle_types=[(8,7,8)],
                  units='real')

IndentationError: unexpected indent (<ipython-input-30-626c485bf635>, line 2)

In [35]:
import numpy as np
def determine_grid(cell, radius):
    ct = cell.transpose()
    a = ct[0]
    b = ct[1]
    c = ct[2]
    al = np.linalg.norm(a)   #vector length
    bl = np.linalg.norm(b)
    cl = np.linalg.norm(c)
    ae = a / al              #unit vectors
    be = b / bl
    ce = c / cl
    ad = np.dot(ae,np.cross(be,ce)) #distance to the bc plane
    bd = np.dot(be,np.cross(ce,ae))
    cd = np.dot(ce,np.cross(ae,be))
    ax = radius / ad        # required length of a vector to contain a sphere of radius 
    bx = radius / bd
    cx = radius / cd
    gf = np.array([al/ax, bl/bx, cl/cx])  # required number of grid cells
    #print(cell,radius,gf)
    #import sys
    #sys.exit(1)
    return np.floor(gf).astype(int)


In [38]:

determine_grid( nano3_low.cell, 7.0)

array([0, 0, 2])